# Export 2019 Recipients and match Canton

In [105]:
import pandas as pd
import mysql.connector

In [106]:
year = 2019

## Connect

In [107]:
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database = "pharmagelder"
)

## Load Data

In [108]:
df_recipient = pd.read_sql("""SELECT
  rec_id as id,
  rec_name as name,
  rec_address as address,
  rec_location as location,
  rec_type as type,
  trc_name as category,
  sum(tra_value) as value
 FROM transaction
  LEFT JOIN transaction_category ON tra_fk_transaction_category = transaction_category.trc_id
  LEFT JOIN recipient ON tra_fk_recipient = recipient.rec_id
  WHERE tra_year = %s
  GROUP BY tra_fk_recipient, tra_fk_transaction_category
  ORDER BY rec_name DESC""" % year, con=db_connection)

In [117]:
df_verzeichnis = pd.read_csv('Ortschaftenverz.-Rép. Localités.csv')
df_GDENAMK = df_verzeichnis[['KTKZ', 'GDENAMK']].copy().rename(columns={'GDENAMK': 'ORTNAME'})
df_PLZNAMK = df_verzeichnis[['KTKZ', 'PLZNAMK']].copy().rename(columns={'PLZNAMK': 'ORTNAME'})
df_verzeichnis = pd.concat([df_verzeichnis, df_GDENAMK, df_PLZNAMK], sort=False).dropna(axis=0, subset=['ORTNAME'])
df_verzeichnis['ORTNAME'] = df_verzeichnis['ORTNAME'].str.lower()
df_verzeichnis = df_verzeichnis[['KTKZ', 'ORTNAME']]

## Match

In [127]:
#Pivot
df_pivot = pd.pivot_table(df_recipient, values='value', columns='category', index=['id', 'name', 'address', 'location', 'type']).reset_index()
df_pivot['total'] = df_pivot.sum(axis=1)
df_pivot['location_s'] = df_pivot['location'].str.lower()

In [128]:
df_new = df_pivot.merge(df_verzeichnis, how='left', left_on='location_s', right_on='ORTNAME')

In [129]:
len(df_new[df_new.KTKZ.isna()])

107

In [130]:
#df_new[df_new.KTKZ.isna()].groupby('location').count().sort_values('name', ascending=False)

## Export

In [133]:
#Write all
df_pivot = df_new.sort_values(['total'], ascending=False)
writer = pd.ExcelWriter('../../data/99. analyzes/excel/recipients/%s_recipients_canton_guessed.xlsx' % year, options={'encoding':'utf-8'})
df_pivot[df_pivot.type == 'hcp'].to_excel(writer, 'hcp', index=True)
df_pivot[df_pivot.type == 'hco'].to_excel(writer, 'hco', index=True)
writer.save()

print("finito")

finito
